In [1]:
from numpy.core.numeric import NaN
from MCtool.RFilter import gray
from genericpath import exists
from matplotlib import image
import math
import sys
import time

import cv2
from matplotlib import pyplot as plt
from tensorflow.python.keras.backend import dtype
from DeepLearning import LearnAndTest
from Rpkg.Rfund.InputFeature import InputFeature
import datetime
import os
import gc
import tensorflow as tf
import random
import numpy as np
import pandas as pd

from Rpkg.Rfund import ReadFile, WriteFile
from Rpkg.Rmodel import Unet, Mnet

import Filtering

import torch


import DeepLearning
from tensorflow.keras.optimizers import Adam

from Rpkg.Rfund.InputFeature import InputFeature
from Rpkg.Rfund import ReadFile, WriteFile
from Rpkg.Rmodel import Unet, Mnet

from MCtool import RFilter, resultEval
from DeepLearning import save_eval_result

import numpy as np
import cv2
import torch
from transformations import ComposeDouble, FunctionWrapperDouble, create_dense_target, normalize_01
from customdatasets import SegmentationDataSet1
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pathlib
from skimage.transform import resize

#early stopping なし
from unet import UNet
from trainer import Trainer




2024-05-22 16:15:04.315792: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-22 16:15:04.986870: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# 自分の環境設定がうまくいったかどうかを確認しましょう、特にGPUの動作
# Prints the version of PyTorch installed
print('PyTorch Version installed: ' + torch.__version__)

# Prints the version of CUDA associated with the installed PyTorch version
print('CUDA version associated with PyTorch version: ' + torch.version.cuda)

# Prints the version of cuDNN (CUDA Deep Neural Network library) being used by PyTorch
print('Version of cuDNN (CUDA Deep Neural Network library) being used by PyTorch' + str(torch.backends.cudnn.version()))

# Same as the line above
print('CUDA is available: ' + str(torch.cuda.is_available()))

# Returns the number of available CUDA-enabled GPUs
print('Number of GPUs compatible with CUDA:' + str(torch.cuda.device_count()))

# Returns the name of the GPU at index 0
print('Name of the GPU at index 0: '  + str(torch.cuda.get_device_name(0)))

# Returns the index of the current CUDA device being used
print('Current CUDA device index: '  + str(torch.cuda.current_device()))


PyTorch Version installed: 2.3.0+cu121
CUDA version associated with PyTorch version: 12.1
Version of cuDNN (CUDA Deep Neural Network library) being used by PyTorch8902
CUDA is available: False
Number of GPUs compatible with CUDA:1


/home/eric/anaconda3/envs/myenv/lib/python3.11/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [44]:
# ファイル名の先頭部分（prefix）により自動的にファイル名を抽出するアルゴリズム。
# 実際それぞれのファイル名は違うと思うので、必須ではない

""" 

Extracts filenames in directory if they start with the prefix input 


Args/Parameters:

    directory_path (string): The path of the dir (ex: /root/home/Documents/etc)
    
    prefix (string): Prefix of the file name (ex: 'Bo' is a prefix of 'Bone')

Returns:

    sorted_file_names (list of str): File names sorted in ascending order in the dir without extension ex: ['bone1', 'bone2', ...]

Raises:

    SomeError: ...

"""

def file_names_with_prefix(directory_path, prefix):

    # Initialize an empty list to store the file names without extensions
    file_names_without_extension = []

    # Loop through all files in the specified directory
    for filename in os.listdir(directory_path):
        #Checking if the file in loop exists in the directory_path not sure how is this necessary
        #??
        if os.path.isfile(os.path.join(directory_path, filename)):
            # Check if the file name starts with the specified prefix
            if filename.startswith(prefix):
                # Get the file name without extension
                name_without_extension, _ = os.path.splitext(filename)

                # Append the file name (without extension) to the list
                file_names_without_extension.append(name_without_extension)

    # Sort the list of file names without extensions in ascending order
    sorted_file_names = sorted(
        file_names_without_extension,
        key=lambda x: (x.split('-')[0], int(x.split('-')[1]))
    )  # Modify this part based on your file naming convention

    # Now you have a sorted list of file names with the specified prefix and without extensions
    return sorted_file_names


In [8]:
##############################
# Custom Function renames all files in format N1-...
# Only need to run it one time
##############################

import os
from PIL import Image

def rename_images(directory):
    # Get a list of all files in the directory
    files = os.listdir(directory)
    
    # Filter out only the image files
    image_files = [file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
    
    # Sort the image files to ensure consistent naming
    image_files.sort()
    
    # Initialize counter
    counter = 1
    
    # Iterate through the image files
    for filename in image_files:
        # Open the image
        image_path = os.path.join(directory, filename)
        with Image.open(image_path) as img:
            # Rename the image file
            new_filename = f"N4-{counter}" + os.path.splitext(filename)[1]
            new_image_path = os.path.join(directory, new_filename)
            
            # Save the image with the new name
            img.save(new_image_path)
        
        # Increment the counter
        counter += 1

# Specify the directory containing the images
directory_path = "/home/Eric/Downloads/imgNaming/processing"

# Call the function to rename the images
rename_images(directory_path)


FileNotFoundError: [Errno 2] No such file or directory: '/home/Eric/Downloads/imgNaming/processing'

In [ ]:
"/home/Eric/Downloads/TestD"

In [45]:
import pathlib
from pathlib import Path

# ここで、folder名とかPathとか色々設定

# Setting the directory name, path and other settings

# Define the root directory where your project is located
# Defining a Path object for the project's root dir
root_dir = Path(pathlib.Path.cwd())

# result folder name
date_str = '0424-Conv1x1-01'

# Define the directories for different types of data
# Concatenating the root dir to the different dataset dirs
data_dir = str(root_dir / "img_1006/original")
feature_dir = str(root_dir / "img_1006/feature") 
labeled_dir = str(root_dir / "img_1006/labeled")
# annealing_img_dir = str(root_dir / "img_1006/annealing_img") # 焼きなまし法時に使う
# annealing later, original for now
annealing_img_dir = str(root_dir / "img_1006/original")
result_dir = str(root_dir / "result" / date_str)
test_result_dir= str(root_dir / "result_test" / date_str)

# Making directories based on the path string result_dir and test_result_dir
Path(result_dir).mkdir(parents=True, exist_ok=True)
Path(test_result_dir).mkdir(parents=True, exist_ok=True)

# Prints the paths of the dirs
print('Root directory: ' + str(root_dir))
print('Data directory (original dir): ' + str(data_dir))
print('Feature img directory: ' + str(feature_dir))
print('Labeled img directory: ' + str(labeled_dir))
print('Annealing directory: ' + str(annealing_img_dir))
print('Result directory: ' + str(result_dir))
print('Test result directory: ' + str(test_result_dir))

# Defining variables filename list of path str starts with the prefix format
input_train = file_names_with_prefix(data_dir, 'N1-')
input_name_val = file_names_with_prefix(data_dir, 'N2-')
annealing_input_name = file_names_with_prefix(data_dir, 'N3-')
input_train.extend(annealing_input_name)
test_input_name = file_names_with_prefix(data_dir, 'N4-') 

# Prints the each data image name
print(input_train)
print(input_name_val)
print(annealing_input_name)
print(test_input_name)

# Defining a var to store each list length
len_train = len(input_train)
len_val = len(input_name_val)
len_test = len(test_input_name)
len_annealing = len(annealing_input_name)


# print(len(input_train))
# print(len(input_name_val))
# print(len(test_input_name))
# print(len(annealing_input_name))


Root directory: /home/eric/Documents/cervicalResearchIIP
Data directory (original dir): /home/eric/Documents/cervicalResearchIIP/img_1006/original
Feature img directory: /home/eric/Documents/cervicalResearchIIP/img_1006/feature
Labeled img directory: /home/eric/Documents/cervicalResearchIIP/img_1006/labeled
Annealing directory: /home/eric/Documents/cervicalResearchIIP/img_1006/original
Result directory: /home/eric/Documents/cervicalResearchIIP/result/0424-Conv1x1-01
Test result directory: /home/eric/Documents/cervicalResearchIIP/result_test/0424-Conv1x1-01
['N1-1', 'N1-2', 'N1-3', 'N1-4', 'N1-5', 'N1-6', 'N1-7', 'N1-8', 'N1-9', 'N3-1', 'N3-2', 'N3-3', 'N3-4', 'N3-5', 'N3-6', 'N3-7', 'N3-8', 'N3-9']
['N2-1', 'N2-2', 'N2-3', 'N2-4', 'N2-5', 'N2-6', 'N2-7', 'N2-8', 'N2-9']
['N3-1', 'N3-2', 'N3-3', 'N3-4', 'N3-5', 'N3-6', 'N3-7', 'N3-8', 'N3-9']
['N4-1', 'N4-2', 'N4-3', 'N4-4', 'N4-5', 'N4-6', 'N4-7', 'N4-8', 'N4-9']


In [46]:
# 特徴画像の特徴一覧をリストとして取得
inputfeature_list = list(map(str, InputFeature))
print(inputfeature_list)

feature_num = len(inputfeature_list)
print(feature_num)

['GRY_', 'NML1', 'NML2', 'NML3', 'TOP1', 'TOP2', 'TOP3', 'TOP4', 'SBLX', 'SBLY', 'SBLM', 'SBLD', 'SBL1', 'SBL2', 'SBL3', 'SBL4', 'LPL1', 'LPL2', 'MEA1', 'MEA2', 'GAU1', 'GAU2', 'MED1', 'MED2', 'LBP1', 'LBP2', 'LBP3', 'ETC1', 'ETC2', 'STC1', 'STC2', 'HGF_', 'NGP_', 'POS1', 'POS2', 'POS3', 'SOL_', 'EMB1', 'EMB2', 'EMB3', 'KNN1', 'KNN2', 'BLT1', 'BLT2', 'OOO_']
45


In [50]:
## 重み計算なし
def CreateWeightImage(input_number):
    label_dataset = []
    arrDataset = []
    for i in input_number:
        label_path = os.path.join(labeled_dir, f"{i}.png")
        input_originallabel = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
        # commented the binary label because the project has more labels than 2 
        # _, binary_label = cv2.threshold(input_originallabel, 0, 255, cv2.THRESH_BINARY)
        label_dataset.append(input_originallabel)

    print("Number of label images:", len(label_dataset))

    for i in input_number:
        # changed this part from 100 to 256
        dataset_img = np.zeros((256, 256, feature_num), dtype=np.float32)

        for m in range(feature_num):
            feature_img_path = os.path.join(feature_dir, str(i), f"{inputfeature_list[m]}.png")
            input_featureimg = cv2.imread(feature_img_path, cv2.IMREAD_GRAYSCALE)
            dataset_img[:, :, m] = input_featureimg

        arrDataset.append(dataset_img)

    arrDataset = np.array(arrDataset)
    print("dataset shape ", arrDataset.shape)
    return arrDataset, label_dataset


In [12]:
# 重みを基づいて、MC画像を生成する
def CreateWeightImageforShow(weight, input_number, index):
    sum_weight = sum(weight)  # Calculate total weight

    label_dataset = []
    input_dataset = []
    dataset_original = []

    # Read label images
    for i in input_number:
        input_originallabel = cv2.imread(labeled_dir + "/" + str(i) + ".png", flags=0)
        label_dataset.append(input_originallabel)

    print('Weight image for show, label length = ', len(label_dataset))

    # Create output directory
    os.makedirs(test_result_dir + "/weightImage/" + str(index), exist_ok=True)

    # Generate weighted images
    for i in input_number:
        # Create a blank image to store the weighted image, using float type for accumulation
        dataset_img = np.zeros((256, 256, 3), dtype=np.float32)
        input_originalimg = cv2.imread(data_dir + "/" + str(i) + ".png")

        dataset_original.append(input_originalimg)
        for m in range(feature_num):
            input_featureimg = cv2.imread(feature_dir + "/" + str(i) + "/" + inputfeature_list[m] + ".png", cv2.IMREAD_GRAYSCALE)
            # Normalize the feature image
            normalized_feature_img = cv2.normalize(input_featureimg.astype(np.float32), None, 0, 1, cv2.NORM_MINMAX)
            # Accumulate the weighted feature image
            dataset_img += normalized_feature_img[:, :, None] * (weight[m] / sum_weight)  # Convert 2D array to 3D array

        # Normalize the accumulated image to the range 0-255
        dataset_img = cv2.normalize(dataset_img, None, 0, 255, cv2.NORM_MINMAX)
        output_img = dataset_img.astype(np.uint8)  # Convert to uint8

        input_dataset.append(output_img)
        # Write to file
        cv2.imwrite(f"{test_result_dir}/weightImage/{index}/{i}.png", output_img)

    return input_dataset, label_dataset


In [9]:
##############################
# Custom Function generates feature img
##############################
import os
import numpy as np
from PIL import Image

def read_images_from_directory(dir_path):
    IMAGE_SIZE = 256


    """
    Reads all image files from a directory and returns a list of images as NumPy arrays.
    
    :param dir_path: Path to the directory containing the images.
    :param image_size: Size of the images (assumes square images).
    
    :return: img_list: List of images as NumPy arrays.
    """
    img_list = []
    fn_without_list = []

    
    # Iterate through all files in the directory
    for filename in os.listdir(dir_path):
        # Check if the file is an image file (you can add more extensions if needed)
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            # Construct the full path to the image
            img_path = os.path.join(dir_path, filename)
            
            # Open the image and resize it to the desired size
            img = Image.open(img_path).convert('L')
            img = img.resize((IMAGE_SIZE, IMAGE_SIZE))
            
            # Convert the image to a NumPy array 
            img_array = np.asarray(img, dtype=np.float64)
            
            # Append the image array to the list
            img_list.append(img_array)
            # Gets the file name without extension for creating dir 
            filename_without_extension = os.path.splitext(filename)[0]
            fn_without_list.append(filename_without_extension)
    
    return img_list, fn_without_list

img_list, fn_without_list = read_images_from_directory(data_dir)
print(len(fn_without_list))
print(fn_without_list[0])
for fn_wthout, img in zip(fn_without_list, img_list):
    WriteFile.make_folder(feature_dir, '/' + fn_wthout)
    fn_feature_list = []
    filtered_img_list = []
    for feature in InputFeature:
        filtered_img= Filtering.single_image(img, feature)
        filtered_img_list.append(filtered_img)
        fn_feature_list.append(str(feature) + '.png')
    WriteFile.save_images(os.path.join(feature_dir,fn_wthout), fn_feature_list, filtered_img_list)
    print("1 iteration")





36
N4-4
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration
1 iteration


In [51]:
## Test code
input_dataset,label_dataset = CreateWeightImage(input_train)
input_dataset_val,label_dataset_val = CreateWeightImage(input_name_val)


Number of label images: 18
dataset shape  (18, 256, 256, 45)
Number of label images: 9
dataset shape  (9, 256, 256, 45)


In [52]:
def print_model_shapes(model, input_tensor):
    def forward_hook(module, input, output):
        print(f"Layer: {module.__class__.__name__}")
        print(f"Input shape: {str(input[0].shape)}")
        print(f"Output shape: {str(output.shape)}")
        print("-----------------------")

    hooks = []
    for layer in model.children():
        hook = layer.register_forward_hook(forward_hook)
        hooks.append(hook)

    print("Model Architecture:")
    print(model)

    # Pass a dummy input tensor through the model to trigger the forward hooks
    with torch.no_grad():
        model(input_tensor)

    for hook in hooks:
        hook.remove()


In [53]:
# preprocess function
def preprocess(img: np.ndarray):
    img = np.moveaxis(img, -1, 0)  # Change from [H, W, C] to [C, H, W]
    img = normalize_01(img)  # Linear scaling to range [0-1]
    img = np.expand_dims(img, axis=0)  # Add batch dimension [B, C, H, W]
    img = img.astype(np.float32)  # Typecasting to float32
    return img

# postprocess function
def postprocess(img: torch.tensor):
    img = torch.argmax(img, dim = 1)  # Perform argmax to generate 1 channel
    img = img * 255.0
    img = img.cpu().numpy().astype(np.uint8)  # Send to CPU and transform to numpy.ndarray
    # If batch_size > 1, you may need to loop through each batch and save them separately
    # If batch_size == 1, you can remove the batch dimension to save a single image
    img = np.squeeze(img)  # Remove batch dim and channel dim -> [H, W]
    # img = re_normalize(img)  # Scale it to the range [0-255]

    # If your image has multiple channels (C>1), like an RGB image, before saving with cv2.imwrite
    # you need to ensure the channel order is [B, G, R] instead of the common [R, G, B]
    # If C == 1, you can further reduce dimensions -> [H, W]
    if img.shape[0] == 3:  # [C, H, W]
        img = np.transpose(img, (1, 2, 0))  # [H, W, C]
        img = img[:, :, ::-1]  # Convert RGB to BGR
    elif img.shape[0] == 1:  # [C, H, W]
        img = np.squeeze(img, 0)  # [H, W]

    return img


In [54]:
# earlystoppingあり
# numpy形式のまま入力する用改良
# 学習を行い予測結果画像を出力するとこまで
from customdatasets import SegmentationDataSet0
from customdatasets import SegmentationDataSet1
from torch.utils.data import DataLoader

def Learn_EA(input_dataset,label_dataset,input_dataset_val,label_dataset_val,type_number):
    # 引数を追加して保存先を指定するよう改良
    # try_number:何回目の焼きなましかどうか。モデルの保存に使用

    # dataset training
    dataset_train2 = SegmentationDataSet0(
                                        #inputs=dataset_original,
                                        inputs=input_dataset,
                                        targets=label_dataset,
                                        transform=transforms_training)

    # dataloader training
    dataloader_training2 = DataLoader(dataset=dataset_train2,
                                     batch_size=2,
                                     shuffle=False)
    #もとはシャッフルtrue


    batch = next(iter(dataloader_training2))
  
    x, y = batch
    print("x.shape = ", x.shape)
    print("x.min(), x.max() = ", x.min(), x.max())
    print("y.shape = ", y.shape)
    print("torch.unique(y) = ", torch.unique(y))


    
    # dataset training
    dataset_val = SegmentationDataSet0(inputs=input_dataset_val,
                                        targets=label_dataset_val,
                                        transform=transforms_val)
    #書き換え箇所
    dataloader_val = DataLoader(dataset=dataset_val,
                                     batch_size=2,
                                     shuffle=False)
    
    
    
    ###earlystopping あり

    from unet import UNet
    from trainer2 import Trainer2 
    from torch import nn #import torch 
    from pytorchtools import EarlyStopping
    from torch.nn import BCEWithLogitsLoss

    #device
    if torch.cuda.is_available():
        device = torch.device('cuda') 
    else: torch.device('cpu')

    #model
    model = UNet(in_channels=45,
                 out_channels=13,
                 n_blocks=4, 
                 start_filters=32,
                 activation='relu',
                 normalization='batch',
                 conv_mode='same',
                 dim=2,
                 ).to(device)

    # Assuming input_tensor is a sample input tensor with the correct shape (e.g., torch.randn(1, 3, 100, 100))
    input_tensor = torch.randn(1, 45, 100, 100).to(device)  # Adjust the shape as needed
    # print_model_shapes(model, input_tensor)


    #criterion
    criterion = torch.nn.CrossEntropyLoss()
    # criterion = BCEWithLogitsLoss()

    #optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    #trainer
    trainer = Trainer2(model=model, 
                       device=device, 
                       criterion=criterion, 
                       optimizer=optimizer, 
                       training_DataLoader=dataloader_training2,
                       #validation_DataLoader=None, 
                       validation_DataLoader=dataloader_val, 
                       lr_scheduler=None, 
                       epochs=200, ##😺😺😺😺 epoch=0, 
                       notebook=True)
  
    print("=======start training======")
    # start training
    training_losses, validation_losses, lr_rates = trainer.run_trainer()
    print("***************************")

    
    
    #ここがちゃんとESで最適なエポック数のモデルになっているか要検証
    model_dir = os.path.join("model", date_str)
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_name = f"model.pt"
    model_path = os.path.join(model_dir, model_name)
    torch.save(model.state_dict(), model_path)
    print(f"modelname:{model_name}を保存しました")

    model_weights = torch.load(model_path)
    model.load_state_dict(model_weights)
    

    # images = annealing_input_dataset
    images = input_dataset_val

    from inference import predict
    from transformations import normalize_01, re_normalize
    # predict the segmentation maps 
    output = [predict(img, model, preprocess, postprocess, device) for img in images]


    for i in range(len(input_name_val)):
        if(type_number == 0):
            cv2.imwrite(os.path.join(result_dir, f'{input_name_val[i]}.png'), output[i])
        elif(type_number == 1):
            cv2.imwrite(os.path.join(test_result_dir, 'result_original', f'{input_name_val[i]}.png'), output[i])
        elif(type_number == 2):
            cv2.imwrite(os.path.join(test_result_dir, 'result_test', f'{input_name_val[i]}.png'), output[i])



In [55]:
## Test Learn_EA without annealing
# training transformations and augmentations
transforms_training = ComposeDouble([
    FunctionWrapperDouble(create_dense_target, input=False, target=True),
    FunctionWrapperDouble(np.moveaxis, input=True, target=False, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01)
])

#追加箇所ver3
transforms_val = ComposeDouble([
    FunctionWrapperDouble(create_dense_target, input=False, target=True),
    FunctionWrapperDouble(np.moveaxis, input=True, target=False, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01)
])

# random seed
random_seed = 0
input_dataset,label_dataset = CreateWeightImage(input_train)
input_dataset_val,label_dataset_val = CreateWeightImage(input_name_val)


Learn_EA(input_dataset,label_dataset,input_dataset_val,label_dataset_val,0)

Number of label images: 18
dataset shape  (18, 256, 256, 45)
Number of label images: 9
dataset shape  (9, 256, 256, 45)
x.shape =  torch.Size([2, 45, 256, 256])
x.min(), x.max() =  tensor(0.) tensor(1.)
y.shape =  torch.Size([2, 256, 256])
torch.unique(y) =  tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [40]:
# Dicecの計算
def cal_Dice(dir, input_name):
    # change the label count as your preference
    Dice = [0] * 2
    Count1 = [0] * 2 #予測結果の各ラベルの要素数
    Count2 = [0] * 2 #ラベル画像の
    Count3 = [0] * 2 #正解した画素数
    unique_label = set()
    for index in range(len(input_name)):
        print('index = ', index)

        img1 = cv2.imread(dir + '/' + input_name[index] + ".png", cv2.IMREAD_GRAYSCALE)
        img2 = cv2.imread(labeled_dir + '/' + input_name[index] + ".png", cv2.IMREAD_GRAYSCALE)
        # _, img2 = cv2.threshold(img2, 0, 255, cv2.THRESH_BINARY)
        print("予測画像:", dir  + '/' + input_name[index] + ".png")
        print("テストラベル:", labeled_dir + '/' + input_name[index] + ".png")
        
        # change the image array size to your need
        for n in range(255):
            for l in range(255):
                value = img1[n,l]
                unique_label.add(value)
                if value == 0:
                    Count1[0] += 1
                else:
                    Count1[1] = Count1[1] + 1
                # Count1[value] = Count1[value] + 1
                value2 = img2[n,l]
                if value2 == 0:
                    Count2[0] = Count2[0] + 1
                else:
                    Count2[1] = Count2[1] + 1
                # Count2[value2] = Count2[value2] + 1

                if(img1[n][l] == img2[n][l]):
                    if value == 0:
                        Count3[0] = Count3[0] + 1
                    else:
                        Count3[1] = Count3[1] + 1
                    # Count3[value] = Count3[value] + 1


    for i in range(2):
        if(Count1[i]+Count2[i] != 0):
            Dice[i] = (2*Count3[i])/(Count1[i] + Count2[i])
    print(unique_label)
    print('Count1 = ', Count1)
    print('Count2 = ', Count2)
    print('Count3 = ', Count3)
    print('Dice = ', Dice)

    return Dice


In [41]:
## Test of 1x1conv

# device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    torch.device('cpu')

model = UNet(in_channels=45,  # check the parameters
             out_channels=13,
             n_blocks=4,
             start_filters=32,
             activation='relu',
             normalization='batch',
             conv_mode='same',
             dim=2).to(device)

model_dir = os.path.join("model", date_str)
print("model_dir = ", model_dir)
model_path = os.path.join(model_dir, "model.pt") # load the model
model_weights = torch.load(model_path)
model.load_state_dict(model_weights)

model.eval()

# create test dataset
test_input_dataset, _ = CreateWeightImage(test_input_name) 

# convert to torch
test_input_tensor = torch.from_numpy(test_input_dataset).float().to(device)

from inference import predict
from transformations import normalize_01, re_normalize
# predict
output = [predict(img, model, preprocess, postprocess, device) for img in test_input_dataset]

# save images
for i, output_img in enumerate(output):
    image = np.ones((255, 255, 3), dtype=np.uint8) * 255
    labels = np.unique(output_img)
    for x in range(255):
            for y in range(255):
                # Get the pixel value (brightness)
                pixel = output_img[x][y]
               #  print(type(output_img))
                if pixel == labels[0]:
                     image[x, y] = [0,0,0]
                elif pixel == labels[1]:
                     image[x, y] = [2, 247, 68]
                elif pixel == labels[2]:
                     image[x, y] = [255, 0, 38]
                elif pixel == labels[3]:
                     image[x, y] = [234, 5, 250]
                elif pixel == labels[4]:
                     image[x, y] = [21, 0, 255]
                elif pixel == labels[5]:
                     image[x, y] = [0, 220, 245]
                elif pixel == labels[6]:
                     image[x, y] = [255, 98, 0]
                elif pixel == labels[7]:
                     image[x, y] = [0, 174, 255]
                elif pixel == labels[8]:
                     image[x, y] = [255, 0, 174]
                elif pixel == labels[9]:
                     image[x, y] = [255, 255, 255]
                elif pixel == labels[10]:
                     image[x, y] = [198, 119, 74]
                elif pixel == labels[11]:
                     image[x, y] = [251, 159, 164]
                elif pixel == labels[12]:
                     image[x, y] = [205, 240, 7]
                else:
                     print("There are more labels!")
    output_img = image
    cv2.imwrite(os.path.join(test_result_dir, f"{test_input_name[i]}.png"), output_img)  # 根据需要调整文件路径和命名
    #print(valRange)


Dice = cal_Dice(test_result_dir, test_input_name)

df = pd.DataFrame(Dice)
df = df.T
df.to_csv(test_result_dir + "/Dice.csv", mode='a', header=False) # Diceの結果をcsvに追加



RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
